In [1]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

import tensorflow as tf

import gpflow
from gpflow.utilities import read_values, multiple_assign, set_trainable
from gpflow.inducing_variables import InducingPoints
import gpmaniflow
from gpmaniflow.models.SVGP import SVGP

import gpmaniflow.samplers
from gpmaniflow.samplers import initialize_sampler
from gpmaniflow.curves import BezierCurve

In [70]:
X = tf.random.normal(shape = [6,3])
X

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[ 0.21203633,  0.26022735,  0.33278173],
       [-1.2920737 , -1.0348426 ,  0.9188523 ],
       [-0.24700649,  0.4879749 , -0.93327165],
       [-0.14512593,  0.18932338, -1.0878857 ],
       [-1.4015514 ,  0.43583226, -0.15929765],
       [ 0.81644213,  1.115221  ,  0.4739572 ]], dtype=float32)>

In [71]:
I = range(6)
I

range(0, 6)

In [72]:
out1, out2 = tf.meshgrid(I,tf.transpose(I))
out = tf.concat([tf.reshape(out2,(-1,1)),tf.reshape(out1,(-1,1))],axis=-1)
out = out[out[:,0] < out[:, 1]]
out # All pairwise indices

<tf.Tensor: shape=(15, 2), dtype=int32, numpy=
array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [0, 5],
       [1, 2],
       [1, 3],
       [1, 4],
       [1, 5],
       [2, 3],
       [2, 4],
       [2, 5],
       [3, 4],
       [3, 5],
       [4, 5]])>

In [88]:
res = tf.gather(X, out)
res = tf.expand_dims(res, 2)
res.shape
len(X)

6

In [87]:
from gpflow.utilities.ops import square_distance
Y = square_distance(X, X)
Y = tf.gather_nd(Y,out)

(6, 6)
tf.Tensor(
[[0.00000000e+00 4.28303194e+00 1.86548030e+00 2.15088844e+00
  2.87664461e+00 1.11625099e+00]
 [4.28303194e+00 0.00000000e+00 6.84150219e+00 6.84106922e+00
  3.33727765e+00 9.26654434e+00]
 [1.86548030e+00 6.84150219e+00 0.00000000e+00 1.23477936e-01
  1.93472874e+00 3.50465369e+00]
 [2.15088844e+00 6.84106922e+00 1.23477936e-01 2.38418579e-07
  2.50164771e+00 4.22125292e+00]
 [2.87664461e+00 3.33727765e+00 1.93472874e+00 2.50164771e+00
  0.00000000e+00 5.78207588e+00]
 [1.11625099e+00 9.26654434e+00 3.50465369e+00 4.22125292e+00
  5.78207588e+00 0.00000000e+00]], shape=(6, 6), dtype=float32)


<tf.Tensor: shape=(15,), dtype=float32, numpy=
array([4.283032  , 1.8654803 , 2.1508884 , 2.8766446 , 1.116251  ,
       6.841502  , 6.841069  , 3.3372777 , 9.266544  , 0.12347794,
       1.9347287 , 3.5046537 , 2.5016477 , 4.221253  , 5.782076  ],
      dtype=float32)>